In [2]:
!pip install tensorflow numpy matplotlib

  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached opt_einsum-3.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached termcolor-2.4.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached namex-0.0.8-py3-none-any.whl.metadata (246 bytes)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 kB 1.3 MB/s eta 0:00:00a 0:00:01
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.2/236.2 MB 8.5 MB/s eta 0:00:00:00:0100:01
Using cached absl_py-2.1.0-py3-none-any.whl (133 kB)
Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
Using cached flatbuffers-24.3.25-py2.py3-none-any.whl (26 kB)
Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt

# Bước 3: Chuẩn bị dữ liệu
# Sử dụng tập dữ liệu MNIST làm ví dụ
(x_train, _), (x_test, _) = tf.keras.datasets.mnist.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Reshape lại dữ liệu để phù hợp với mô hình VAE
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))

# Bước 4: Định nghĩa mô hình VAE
latent_dim = 2  # Kích thước của không gian tiềm ẩn

# Encoder
def encoder_model():
    inputs = layers.Input(shape=(28, 28, 1))
    x = layers.Conv2D(32, 3, activation='relu', strides=2, padding='same')(inputs)
    x = layers.Conv2D(64, 3, activation='relu', strides=2, padding='same')(x)
    x = layers.Flatten()(x)
    x = layers.Dense(16, activation='relu')(x)
    z_mean = layers.Dense(latent_dim)(x)
    z_log_var = layers.Dense(latent_dim)(x)
    return tf.keras.Model(inputs, [z_mean, z_log_var], name='encoder')

# Sampling layer
def sampling(args):
    z_mean, z_log_var = args
    batch = tf.shape(z_mean)[0]
    dim = tf.shape(z_mean)[1]
    epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon

# Decoder
def decoder_model():
    latent_inputs = layers.Input(shape=(latent_dim,))
    x = layers.Dense(7 * 7 * 64, activation='relu')(latent_inputs)
    x = layers.Reshape((7, 7, 64))(x)
    x = layers.Conv2DTranspose(64, 3, activation='relu', strides=2, padding='same')(x)
    x = layers.Conv2DTranspose(32, 3, activation='relu', strides=2, padding='same')(x)
    outputs = layers.Conv2DTranspose(1, 3, activation='sigmoid', padding='same')(x)
    return tf.keras.Model(latent_inputs, outputs, name='decoder')

# VAE model
class VAE(tf.keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder

    def call(self, inputs):
        z_mean, z_log_var = self.encoder(inputs)
        z = sampling([z_mean, z_log_var])
        reconstructed = self.decoder(z)
        kl_loss = -0.5 * tf.reduce_mean(tf.reduce_sum(
            1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), axis=-1))
        self.add_loss(kl_loss)
        return reconstructed

# Khởi tạo encoder và decoder
encoder = encoder_model()
decoder = decoder_model()

# Khởi tạo VAE
vae = VAE(encoder, decoder)

# Bước 5: Huấn luyện mô hình
vae.compile(optimizer='adam', loss=tf.keras.losses.MeanSquaredError())
vae.fit(x_train, x_train, epochs=20, batch_size=128, validation_data=(x_test, x_test))

# Bước 6: Sử dụng mô hình VAE để phát hiện bất thường
# Tính lỗi tái tạo (reconstruction error) trên tập dữ liệu kiểm tra
reconstructions = vae.predict(x_test)
reconstruction_errors = np.mean(np.square(x_test - reconstructions), axis=(1, 2, 3))

# Thiết lập ngưỡng để phát hiện bất thường
threshold = np.percentile(reconstruction_errors, 95)  # Giả sử ngưỡng là 95%
anomalies = reconstruction_errors > threshold

print(f'Ngưỡng phát hiện bất thường: {threshold}')
print(f'Số lượng bất thường được phát hiện: {np.sum(anomalies)}')

# Bước 7: Hiển thị một vài ví dụ về các điểm bất thường
n = 10
plt.figure(figsize=(20, 4))
for i in range(n):
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[anomalies][i].reshape(28, 28), cmap='gray')
    plt.title("Anomaly")
    plt.axis("off")

    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(reconstructions[anomalies][i].reshape(28, 28), cmap='gray')
    plt.title("Reconstruction")
    plt.axis("off")
plt.show()


ModuleNotFoundError: No module named 'tensorflow'